In [1]:
#This code finds the mean AR freq
#Done for each algorithm individually 
 
import numpy as np
import glob
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import warnings
import dask.array as da
import dask
import matplotlib as mpl
import xarray as xr
import numpy.ma as ma

In [24]:
#Reading data for one algorithm at a time
#Path='/N/slate/dkamnani/Cascade_Bard/Data/*.nc4'
#Path='/N/slate/dkamnani/Mundhenk/*.nc4'
#Path='/N/slate/dkamnani/ClimateNet_DL/*.nc4'
#Path='/N/slate/dkamnani/reid250/*.nc4'
#Path='/N/slate/dkamnani/guan_waliser/*.nc4'
#DS=xr.open_mfdataset(Path,chunks={'time':2928})
Path='/N/slate/dkamnani/IVT_MEERA/*.nc'
#Path='/N/scratch/dkamnani/vIVT/Final/*.nc'
print(Path)
DS=xr.open_mfdataset(Path, parallel=True)
#Freq=DS.ar_binary_tag
Freq=DS.IVT
#Freq=da.from_array(Freq)
#print("The initial dataset is")
print(Freq)

/N/slate/dkamnani/IVT_MEERA/*.nc
<xarray.DataArray 'IVT' (time: 111040, lat: 361, lon: 576)>
dask.array<concatenate, shape=(111040, 361, 576), dtype=float32, chunksize=(2928, 361, 576), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 ... 2017-12-31T21:00:00
  * lon      (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
Attributes:
    standard_name:   eastward_wind
    long_name:       Integrated Water Vapor Transport
    units:           kg/m/s
    time:            0
    fmissing_value:  1000000000000000.0
    vmax:            1000000000000000.0
    vmin:            -1000000000000000.0
    _ChunkSizes:     [  1   1  91 144]
    lev:             1000.0
    short_name:      IVT


In [25]:
#Calculating the seasonal averages

#setting the months for each season
seasons = dict(
    DJF = [12,  1,  2],
    MAM = [ 3,  4,  5],
    JJA = [ 6,  7,  8],
    SON = [ 9, 10, 11])
# initialize the seasonal mean dataset as a list
seasonal_mean_dataset = []
# loop over seasons
for season in seasons:
    print(f"Working on {season}")
    
    # determine the month associated with each timestep
    data_month = Freq['time.month']
    
    # determine whether each time step is within the current season (True=yes, False=no)
    in_season = np.in1d(data_month, seasons[season])
    #print(in_season)
    
    # subset the data so that only includes months in the current season
    Freq_subset = Freq.sel(time = Freq['time'][in_season])
    #print(Freq['time'][in_season])
    
    # set the year values by which we'll group the data
    season_years = Freq['time.year']
    #print(season_years)
    
    # for DJF, D and JF have different years, meaning they would be grouped incorrectly
    # (Jan 2020, Feb 2020... and Dec 2020 would all be grouped into the same year, whereas we want
    #  Dec 2019 to be grouped instead)
    # The following artificially shifts the year for december to account for this, so that Dec 2019 becomes 'Dec 2020'
    if season == "DJF":
        # shift data in all decembers ahead by one year so they're grouped with the correct Jan and Feb
        season_years[data_month == 12] = season_years[data_month == 12] + 1
        
    # subset the the season_year variable so we only include months in the current season
    season_years = season_years.sel(time = Freq['time'][in_season])
        
        
    # group the data by year (accounting for seasons that span a year change)
    year_group = Freq_subset.groupby(season_years)
    
    # calculate the mean
    seasonal_mean = year_group.mean(dim = "time")*100
    
    # append the seasonal mean dataset to the list
    seasonal_mean_dataset.append(seasonal_mean)
    
# create an array of seasons
season_array = xr.DataArray(list(seasons), dims = ('season',))
#print(season_array)

# create a new dataset from the list of seasonal means (overwriting the list)
seasonal_mean_dataset = xr.concat(seasonal_mean_dataset, dim = season_array)
    
# show the dataset
print(seasonal_mean_dataset)


Working on DJF
Working on MAM
Working on JJA
Working on SON
<xarray.DataArray 'IVT' (season: 4, year: 39, lat: 361, lon: 576)>
dask.array<concatenate, shape=(4, 39, 361, 576), dtype=float32, chunksize=(1, 1, 361, 576), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * year     (year) int64 1980 1981 1982 1983 1984 ... 2014 2015 2016 2017 2018
  * season   (season) <U3 'DJF' 'MAM' 'JJA' 'SON'


In [26]:
#slicing the data
#removing the first and last year because we need December from the previous year and Jan and Feb from the next
seasonal_mean_1=seasonal_mean_dataset.isel(year=slice(1,37))
print(seasonal_mean_1)

#Saving lat and lon to file
lat=seasonal_mean_1.lat
lon=seasonal_mean_1.lon
lat=np.save('lat.npy',lat)
lon=np.save('lon.npy',lon)


<xarray.DataArray 'IVT' (season: 4, year: 36, lat: 361, lon: 576)>
dask.array<getitem, shape=(4, 36, 361, 576), dtype=float32, chunksize=(1, 1, 361, 576), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * year     (year) int64 1981 1982 1983 1984 1985 ... 2012 2013 2014 2015 2016
  * season   (season) <U3 'DJF' 'MAM' 'JJA' 'SON'


In [27]:
mean_ar=seasonal_mean_1.mean(dim='season',skipna=True)
print(mean_ar)

<xarray.DataArray 'IVT' (year: 36, lat: 361, lon: 576)>
dask.array<mean_agg-aggregate, shape=(36, 361, 576), dtype=float32, chunksize=(1, 361, 576), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * year     (year) int64 1981 1982 1983 1984 1985 ... 2012 2013 2014 2015 2016


In [28]:
mean_ar_yearly=mean_ar.mean(dim='year',skipna=True)
print(mean_ar_yearly)


<xarray.DataArray 'IVT' (lat: 361, lon: 576)>
dask.array<mean_agg-aggregate, shape=(361, 576), dtype=float32, chunksize=(361, 576), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0


In [29]:
print(mean_ar_yearly.max().values)

40790.64


In [30]:
#Writing to file
mean_ar_yearly.to_netcdf("Mean_AR_IVT.nc")

In [ ]:
#Writing to file
mean_ar.to_netcdf('Mean_AR_GW.nc')